In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
import numpy as np
from numpy.linalg import inv

In [5]:
# load the original link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [6]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [7]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [8]:
len(link_day_minute_Apr_list)

60480

In [9]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [10]:
np.size(x, 0), np.size(x, 1)

(24, 2400)

In [11]:
x[:,1]

matrix([[ 340870.69418863],
        [ 358486.38585181],
        [ 639230.39632583],
        [ 664588.27655425],
        [ 169784.36832265],
        [ 233410.20644246],
        [ 419817.0297148 ],
        [ 410135.43481856],
        [ 393498.46510653],
        [ 420136.97259902],
        [ 430028.83789741],
        [ 266240.5339992 ],
        [ 103337.77220933],
        [  95613.56467784],
        [ 169261.09059214],
        [ 129407.53557805],
        [ 216161.04998767],
        [ 214551.0093072 ],
        [ 407087.39149114],
        [ 352159.04364636],
        [ 498148.97401147],
        [ 572200.77348568],
        [ 141588.80516519],
        [ 136945.32244788]])

In [12]:
from gurobipy import *

def flow_conservation_adjustment(y_0):

    L = len(y_0)  # dimension of flow vector x
    assert(L == 24)

    # y_0 = x[:,1]  # initial flow vector


    model = Model("Flow_conservation_adjustment")

    y = []
    for l in range(L):
        y.append(model.addVar(name='y_' + str(l)))

    model.update() 

    # Set objective: ||y-y_0||^2
    obj = 0
    for l in range(L):
        obj += (y[l] - y_0[l]) * (y[l] - y_0[l])
    model.setObjective(obj)

    # Add nonnegative constraint: y >= 0
    for l in range(L):
        model.addConstr(y[l] >= 0)
    # Add flow conservation constraints
    model.addConstr(y[1] + y[3] == y[0] + y[2])
    model.addConstr(y[0] + y[5] + y[7] == y[1] + y[4] + y[6])
    model.addConstr(y[2] + y[4] + y[9] + y[11] == y[3] + y[5] + y[8] + y[10])
    model.addConstr(y[6] + y[13] + y[17] == y[7] + y[12] + y[16])
    model.addConstr(y[8] + y[12] + y[15] + y[19] == y[9] + y[13] + y[14] + y[18])
    model.addConstr(y[10] + y[14] + y[21] == y[11] + y[15] + y[20])
    model.addConstr(y[18] + y[20] + y[23] == y[19] + y[21] + y[22])
    model.addConstr(y[16] + y[22] == y[17] + y[23])

    model.update() 

    model.setParam('OutputFlag', False)
    model.optimize()

    y = []
    for v in model.getVars():
        # print('%s %g' % (v.varName, v.x))
        y.append(v.x)
    # print('Obj: %g' % obj.getValue())
    return y

In [13]:
y_0 = x[:,1]  # initial flow vector
y = flow_conservation_adjustment(y_0)

In [14]:
y[0] + y[2], y[1] + y[3]

(1001587.8764602572, 1001587.8764602572)

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [39]:
y

[341942.83822016924,
 357414.2418202704,
 659645.038240088,
 644173.6346399868,
 189126.8662053739,
 214067.70855973492,
 419710.95164380973,
 410241.51288954995,
 380499.3411431444,
 433136.0965624032,
 369718.34428080556,
 326551.02761580655,
 109787.22419965219,
 89164.11268751879,
 121949.72093892723,
 176718.905231266,
 209779.19326849905,
 220932.86602637265,
 391000.98776135704,
 368245.4473761436,
 529373.9399349082,
 540975.807562248,
 144843.9001854697,
 133690.22742759614]